# Controllable Music Generation with MusicGen and OpenVINO

MusicGen is a single-stage auto-regressive Transformer model capable of generating high-quality music samples conditioned on text descriptions or audio prompts. The text prompt is passed to a text encoder model (T5) to obtain a sequence of hidden-state representations. These hidden states are fed to MusicGen, which predicts discrete audio tokens (audio codes). Finally, audio tokens are then decoded using an audio compression model (EnCodec) to recover the audio waveform.

![pipeline](https://)

[The MusicGen model](https://arxiv.org/abs/2306.05284) does not require a self-supervised semantic representation of the text/audio prompts; it operates over several streams of compressed discrete music representation with efficient token interleaving patterns, thus eliminating the need to cascade multiple models to predict a set of codebooks (e.g. hierarchically or upsampling). Unlike prior models addressing music generation, it is able to generate all the codebooks in a single forward pass.

In this tutorial, we consider how to run the MusicGen model using OpenVINO.

We will use a model implementation from the [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) library. 

The tutorial consists of the following steps:

- Install prerequisites
- Download the MusicGen Small model from a public source using the [Hugging Face Hub](https://huggingface.co/models?sort=downloads&search=facebook%2Fmusicgen-).
- Run the text-conditioned music generation pipeline
- Convert three models backing the MusicGen pipeline
- Run the music generation pipeline again using OpenVINO

## Prerequisites

### Install requirements

In [1]:
!pip install -q -U --quiet pip
!pip install -q --pre openvino
!pip install -q torch onnx gradio
!pip install -q git+https://github.com/huggingface/transformers.git datasets[audio]

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be fou

### Imports

In [1]:
from collections import namedtuple
import gc
from pathlib import Path
from typing import Optional, Tuple
import warnings

from IPython.display import Audio
from openvino import Core, convert_model, PartialShape, save_model, Type
import torch
from torch.jit import TracerWarning
from transformers import AutoProcessor, MusicgenForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutputWithPastAndCrossAttentions, CausalLMOutputWithCrossAttentions

# Ignore tracing warnings
warnings.filterwarnings("ignore", category=TracerWarning)

## MusicGen in HF Transformers

To work with MusicGen by Meta AI, we will use [Hugging Face Transformers package](https://github.com/huggingface/transformers). Transformers package exposes the `MusicgenForConditionalGeneration` class, simplifying the model instantiation and weights loading. The code below demonstrates how to create a `MusicgenForConditionalGeneration` and generate a text-conditioned music sample.

In [25]:
# Load the pipeline
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small", torchscript=True, return_dict=False)

In [3]:
device = "cpu"
sample_length = 10  # seconds
n_tokens = sample_length * model.config.audio_encoder.frame_rate
sampling_rate = model.config.audio_encoder.sampling_rate
print(n_tokens, 'tokens will be generated')
print('Sampling rate is', sampling_rate, 'Hz')

model.to(device)
model.eval();

500 tokens will be generated
Sampling rate is 32000 Hz


### Original Pipeline Inference

In [24]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth"],
    return_tensors="pt",
)

audio_values = model.generate(**inputs.to(device), do_sample=True, guidance_scale=3, max_new_tokens=n_tokens)

Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)

KeyboardInterrupt: 

## Convert models to OpenVINO Intermediate representation (IR) format
Model conversion API enables direct conversion of PyTorch models. We will utilize the `openvino.ovc.convert_model` method to acquire OpenVINO IR versions of the models. The method requires a model object and example input for model tracing. Under the hood,the  converter will use the PyTorch JIT compiler, to build a frozen model graph.

The pipeline consists of three important parts:

 - The T5 Text Encohat translates user prompts to vectors in the latent space that the next model - MusicGen decoder can utilize.
 - A MusicGen Language Model that auto-regressively generates audio tokens (codes).
 - EnCodec model (we will use only the decoder partof), it is used decode the audio waveform from the audio tokens predicted by the MusicGen LM

Let us convert each model step by step.

### 0. Set Up Variables

In [5]:
models_dir = Path("./models")
t5_ir_path = models_dir / "t5.xml"
musicgen_0_ir_path = models_dir / "mg_0.xml"
musicgen_ir_path = models_dir / "mg.xml"
audio_decoder_onnx_path = models_dir / 'encodec.onnx'
audio_decoder_ir_path = models_dir / "encodec.xml"

### 1. Convert Text Encoder

The text encoder is responsible for converting the input prompt, such as "90s rock song with loud guitars and heavy drums" into an embedding space that can be fed to the next model. Typically, it is a transformer-based encoder that maps a sequence of input tokens to a sequence of text embeddings.

The input for the text encoder consists of a tensor `input_ids`, which contains token indices from the text processed by the token,izer and `attention_mask` that we will ignore as we will process one prompt at a time and it will just consist of ones.

In [6]:
if not t5_ir_path.exists():
    t5_ov = convert_model(model.text_encoder, example_input={'input_ids': inputs['input_ids']})

    save_model(t5_ov, t5_ir_path)
    del t5_ov
    gc.collect()

### 2. Convert MusicGen Language Model

This model is the central part of the whole pipeline, it takes the embedded text representation and generates audio codes that can be then decoded into actual music. The model outputs several streams of audio codes - tokens sampled from the pre-trained codebooks representing music efficiently with a lower framerate. The model employs innovative codes intervaling strategy, that makes single-stage generation possible.

On the 0th generation step the model accepts `input_ids` representing the indices of audio codes, `encoder_hidden_states` and `encoder_attention_mask` that were provided by the text encoder.

In [7]:
# Set model config `torchscript` to True, so the model returns a tuple as output
model.decoder.config.torchscript = True

if not musicgen_0_ir_path.exists():
    decoder_input = {
        'input_ids': torch.ones(8, 1, dtype=torch.int64),
        'encoder_hidden_states':  torch.ones(2, 12, 1024, dtype=torch.float32),
        'encoder_attention_mask':  torch.ones(2, 12, dtype=torch.int64),
    }
    mg_ov_0_step = convert_model(model.decoder, example_input=decoder_input)

    save_model(mg_ov_0_step, musicgen_0_ir_path)
    del mg_ov_0_step
    gc.collect()

On further iterations, the model is also provided with a `past_key_values` argument that contains previous outputs of the attention block, it allows us to save on computations.
But for us, it means that the signature of the model's `forward` method changed. Models in OpenVINO IR have frozen calculation graphs and do not allow optional arguments, that is why the MusicGen model must be converted a second time, with an increased number of inputs.

In [8]:
# Add additional argument to the example_input dict
if not musicgen_ir_path.exists():
    # Add `past_key_values` to the converted model signature  
    decoder_input['past_key_values'] = tuple(
        [(
            torch.ones(2, 16, 1, 64, dtype=torch.float32),
            torch.ones(2, 16, 1, 64, dtype=torch.float32),
            torch.ones(2, 16, 12, 64, dtype=torch.float32),
            torch.ones(2, 16, 12, 64, dtype=torch.float32),
        )]*24
    )

    mg_ov = convert_model(model.decoder, example_input=decoder_input)

Moreover, the `past_key_values` argument is passed as `Tuple[Tuple[torch.tensor]]` which is a hard call for the converter. The converter flattens these tuples, but have hard time identifying the correct shapes for the tensors. The code below goes over raveled `past_key_values`-related model inputs and sets the correct shape and type for them.

In [9]:
if not musicgen_ir_path.exists():
    for input in mg_ov.inputs[3:]:
        input.get_node().set_partial_shape(PartialShape([-1, 16, -1, 64]))
        input.get_node().set_element_type(Type.f32)
    
    mg_ov.validate_nodes_and_infer_types()

    save_model(mg_ov, musicgen_ir_path)
    del mg_ov
    gc.collect()

### 3. Convert Audio Decoder

The audio decoder which is a part of the EnCodec model is used to recover the audio waveform from the audio tokens predicted by the MusicGen decoder. To learn more about the model please refer to the corresponding [OpenVINO example](https://github.com/openvinotoolkit/openvino_notebooks/tree/main/notebooks/234-encodec-audio-compression).

The audio decoder computation graph contains an LSTM sub-network which currently should be converted through ONNX.
To do this we create a wrapper class with its `forward()` method calling `encodec.decode()`.

In [38]:
if not audio_decoder_onnx_path.exists():
    class AudioDecoder(torch.nn.Module):
        def __init__(self, model):
            super().__init__()
            self.model = model
    
        def forward(self, output_ids):
            return self.model.decode(output_ids, [None])

    audio_decoder_input = {'output_ids': torch.ones(1, 1, 4, 253, dtype=torch.int64),}

    with torch.no_grad():
        torch.onnx.export(
            model=AudioDecoder(model.audio_encoder), 
            args=audio_decoder_input,
            f=audio_decoder_onnx_path,
            input_names=['output_ids',],
            output_names=['decoded_audio'],
            dynamic_axes={'output_ids': {3: 'sequence_length'},
                          'decoded_audio': {2: 'audio_values'}
                         }
        )

/home/idavidyu/.virtualenvs/test/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4476: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [50]:
encoder_input = {'output_ids': torch.ones(1, 1, 4, 500, dtype=torch.int64),}
core.compile_model(str(audio_decoder_onnx_path))(list(encoder_input.values()))[0].shape

(1, 1, 161920)

In [47]:
model.audio_encoder.decode(torch.ones(1, 1, 4, 500, dtype=torch.int64), [None]).audio_values.shape

torch.Size([1, 1, 320000])

In [35]:
161920 / 253 / 4

160.0

In [48]:
320000 / 500

640.0

In [19]:
# Now we can convert the model to OpenVINO IR
if not audio_decoder_ir_path.exists():
    audio_decoder_ov =  convert_model(str(audio_decoder_onnx_path))

    save_model(audio_decoder_ov, audio_decoder_ir_path)
    del audio_decoder_ov
    gc.collect()

## Embeddingthe converted models into the original pipeline

In [20]:
core = Core()

In [21]:
class TextEncoderWrapper(torch.nn.Module):
    def __init__(self, encoder_ir, config):
        super().__init__()
        self.encoder = core.compile_model(encoder_ir)
        self.config = config

    def forward(self, input_ids, **kwargs):
        last_hidden_state = self.encoder(input_ids)[self.encoder.outputs[0]]
        last_hidden_state = torch.tensor(last_hidden_state)
        return BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=last_hidden_state)


class MusicGenWrapper(torch.nn.Module):
    def __init__(self, music_gen_lm_0_ir, music_gen_lm_ir, config, num_codebooks, build_delay_pattern_mask,
                 apply_delay_pattern_mask):
        super().__init__()
        self.music_gen_lm_0 = core.compile_model(music_gen_lm_0_ir)
        self.music_gen_lm = core.compile_model(music_gen_lm_ir)
        self.config = config
        self.num_codebooks = num_codebooks
        self.build_delay_pattern_mask = build_delay_pattern_mask
        self.apply_delay_pattern_mask = apply_delay_pattern_mask

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        encoder_hidden_states: torch.FloatTensor = None,
        encoder_attention_mask: torch.LongTensor = None,
        past_key_values: Optional[Tuple[torch.FloatTensor]] = None,
        **kwargs
    ):
        if past_key_values is None:
            model = self.music_gen_lm_0
            arguments = (input_ids, encoder_hidden_states, encoder_attention_mask)
        else:
            model = self.music_gen_lm
            arguments = (input_ids, encoder_hidden_states, encoder_attention_mask, *past_key_values)
            
        output = model(arguments)
        return CausalLMOutputWithCrossAttentions(
            logits = torch.tensor(output[model.outputs[0]]),
            past_key_values = tuple([output[model.outputs[i]] for i in range(1, 97)]),
        )

class AudioDecoderWrapper(torch.nn.Module):
    def __init__(self, decoder_ir, config):
        super().__init__()
        self.decoder = core.compile_model(decoder_ir)
        self.config = config
        self.output_type = namedtuple("AudioDecoderOutput", ["audio_values"])

    def decode(self, output_ids, audio_scales):
        output = self.decoder(output_ids)[self.decoder.outputs[0]]
        return self.output_type(audio_values=torch.tensor(output))

In [22]:
text_encode_ov = TextEncoderWrapper(t5_ir_path, model.text_encoder.config)
musicgen_decoder_ov = MusicGenWrapper(
    musicgen_0_ir_path,
    musicgen_ir_path,
    model.decoder.config,
    model.decoder.num_codebooks,
    model.decoder.build_delay_pattern_mask,
    model.decoder.apply_delay_pattern_mask
)
audio_encoder_ov = AudioDecoderWrapper(audio_decoder_ir_path, model.audio_encoder.config)

del model.text_encoder
del model.decoder
del model.audio_encoder
gc.collect()

model.text_encoder = text_encode_ov
model.decoder = musicgen_decoder_ov
model.audio_encoder = audio_encoder_ov

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
inputs = processor(
    text=["80s pop track with bassy drums and synth",],
    padding=True,
    return_tensors="pt",
)
audio_values = model.generate(**inputs.to(device), do_sample=True, guidance_scale=3, max_new_tokens=n_tokens)
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)

RuntimeError: Exception from src/inference/src/infer_request.cpp:206:
Check 'inputs.size() == 1' failed at src/inference/src/infer_request.cpp:208:
get_input_tensor() must be called on a function with exactly one parameter.

